In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

In [2]:
!pip install pyarrow

In [3]:
pd.__version__

'1.4.2'

In [4]:
# df1 = pd.read_parquet('../data/yellow_tripdata_2023-01.parquet')
# df2 = pd.read_parquet('../data/yellow_tripdata_2023-02.parquet')
# df1.head()

In [5]:
# df1.columns

In [6]:
def read_clean_df(filename):
    df = pd.read_parquet(filename)
    
    df['tpep_dropoff_datetime'] = pd.to_datetime(df['tpep_dropoff_datetime'])
    df['tpep_pickup_datetime'] = pd.to_datetime(df['tpep_pickup_datetime'])
    
    df['duration'] = df['tpep_dropoff_datetime'] - df['tpep_pickup_datetime']
    df['duration'] = df['duration'].apply(lambda x: x.total_seconds() / 60)
    
    df = df.loc[((df.duration >= 1) & (df.duration <= 60))]
    
    cat_feats = ['PULocationID', 'DOLocationID']
    df[cat_feats] = df[cat_feats].astype(str)
    
    return df

In [7]:
df1 = read_clean_df('../data/yellow_tripdata_2023-01.parquet')
df2 = read_clean_df('../data/yellow_tripdata_2023-02.parquet')

In [8]:
len(df1.columns)

20

In [9]:
import sklearn
sklearn.__version__

'1.0.2'

In [10]:
df1.dtypes

VendorID                          int64
tpep_pickup_datetime     datetime64[ns]
tpep_dropoff_datetime    datetime64[ns]
passenger_count                 float64
trip_distance                   float64
RatecodeID                      float64
store_and_fwd_flag               object
PULocationID                     object
DOLocationID                     object
payment_type                      int64
fare_amount                     float64
extra                           float64
mta_tax                         float64
tip_amount                      float64
tolls_amount                    float64
improvement_surcharge           float64
total_amount                    float64
congestion_surcharge            float64
airport_fee                     float64
duration                        float64
dtype: object

In [11]:
cat_feats = ['PULocationID', 'DOLocationID']
train_dicts = df1[cat_feats].to_dict(orient='records')

In [12]:
dv = DictVectorizer()
X_train = dv.fit_transform(train_dicts)

In [13]:
X_train

<3009173x515 sparse matrix of type '<class 'numpy.float64'>'
	with 6018346 stored elements in Compressed Sparse Row format>

In [14]:
target = 'duration'
y_train = df1[target].values
y_train

array([ 8.43333333,  6.31666667, 12.75      , ..., 24.51666667,
       13.        , 14.4       ])

In [15]:
lr = LinearRegression()
lr.fit(X_train, y_train)

LinearRegression()

In [16]:
y_pred = lr.predict(X_train)

In [ ]:
sns.distplot(y_pred, label='prediction')
sns.distplot(y_train, label='actual')

plt.legend()

/home/codespace/anaconda3/lib/python3.9/site-packages/seaborn/distributions.py:2619: FutureWarning: `distplot` is a deprecated function and will be removed in a future version. Please adapt your code to use either `displot` (a figure-level function with similar flexibility) or `histplot` (an axes-level function for histograms).
  warnings.warn(msg, FutureWarning)
/home/codespace/anaconda3/lib/python3.9/site-packages/seaborn/distributions.py:2619: FutureWarning: `distplot` is a deprecated function and will be removed in a future version. Please adapt your code to use either `displot` (a figure-level function with similar flexibility) or `histplot` (an axes-level function for histograms).
  warnings.warn(msg, FutureWarning)


In [ ]:
mean_squared_error(y_train, y_pred)

In [ ]:
mean_squared_error(y_train, y_pred)**(.5)

In [ ]:
valid_dicts = df2[cat_feats].to_dict(orient='records')

In [ ]:
X_valid = dv.transform(valid_dicts)

In [ ]:
y_valid = df2[target].values
y_valid

In [ ]:
X_valid

In [ ]:
y_pred_valid = lr.predict(X_valid)

In [ ]:
mean_squared_error(y_valid, y_pred_valid)**(.5)

In [ ]:
sns.distplot(y_pred_valid, label='prediction')
sns.distplot(y_valid, label='actual')

plt.legend()